<b> 
Takes existing BQ PROC TEMPLATE and inserts SQL from [referenced external files defined by "Analysts"] while replacing placeholders with PROC PARAMETERS AND VARIABLES:

    List of features...
    


In [8]:
import sys
import os

if 'Documents\ipynb\dep' not in os.getcwd():
    os.chdir('dep')

from FileOps import fs

In [9]:

#X4_TEMPLATE.sql
#  string to stream 
#  replace placeholders first
#  split by ), and REPLACE 
#    FIRST line [1,-X] with CREATE TEMP TABLE
#    LAST  line , with ;
#  further split [-1] by --%%INSERT%%
#    append [0] + ';'
#    [--%%INSERT%%] with [1]

sqt = fs()
sqStr = sqt.getLinesFromFile('..\kiwi_PROD.sp_process_X4_TEMPLATE.sql')
#Probably need a lines from File?

AttributeError: 'fs' object has no attribute 'getLinesFromFile'

In [7]:
asq = sqStr.split(',')
for i in asq:
    print(i)

CREATE OR REPLACE PROCEDURE kiwi_PROD.sp_process_X4(IN from_date DATE
 IN to_date DATE
 IN debug_mode BOOL) BEGIN    --CALL kiwi_TEST.sp_process_X4('2019-05-17'
 '2019-05-17'
 TRUE);    --DECLARE item DEFAULT (SELECT item FROM dataset1.products LIMIT 1);   DECLARE dt DATE;   DECLARE dset STRING DEFAULT 'kiwi_TEST';      SET dt = from_date;      WHILE dt <= to_date DO     BEGIN        --variables can have different scopes
 so set WHILE scope variables here       DECLARE bq_ts TIMESTAMP;       DECLARE dtString STRING;       DECLARE dtPrefix STRING;       DECLARE proc_instance STRING;       DECLARE dtbl STRING;      --SET (variable_name_1
 variable_name_2
 â€¦
 variable_name_n) =       --(expression_1
 expression_2
 â€¦
 expression_n);        SET bq_ts = @@script.creation_time;       SET dtString = CAST(dt as STRING);       SET dtPrefix = CONCAT('%'
 REPLACE(dtString
 '-'
 '/')
 '%.gz.parquet');       SET proc_instance = CONCAT('kiwi_TEST.sp_process_X4'
 ' '
 dtString); 	         IF debug

In [3]:
if opType == 'Create':    # Template Setup
    sDrop = '/*\nALTER TABLE x.xx SET (SYSTEM_VERSIONING = OFF); \nDROP TABLE x.xx; \nDROP TABLE x.xx_History;\n*/ \n\n'

    s =   ',\n[StartTime] [datetime2](7) GENERATED ALWAYS AS ROW START NOT NULL,\n[EndTime] [datetime2](7) GENERATED ALWAYS AS ROW END NOT NULL,\n'
    s = s + 'CONSTRAINT [PK_x.xx] PRIMARY KEY NONCLUSTERED( [BKEY] ASC ),\nPERIOD FOR SYSTEM_TIME ([StartTime], [EndTime])\n)\n'
    s = s + 'WITH (  SYSTEM_VERSIONING = ON ( HISTORY_TABLE = x.xx_History /*,HISTORY_RETENTION_PERIOD = 3 YEARS*/ )  );'
    sTemporal = s

    sID = '[ID] INT IDENTITY,\n'  #insert after .index('(')
    sPK = 'CONSTRAINT [PK_x.xx] PRIMARY KEY CLUSTERED( [BKEY] ASC ),\n'
    sBK = f'CREATE UNIQUE INDEX {sBKpattern} ON x.xx( [BKEY] ASC ) \n'

elif opType == 'Alter':
    s =   'ALTER TABLE x.xx ADD StartTime DATETIME2, EndTime DATETIME2\n'
    s = s + "UPDATE x.xx SET StartTime = '19000101 00:00:00.0000000', EndTime = '99991231 23:59:59.9999999'\n"
    s = s + 'ALTER TABLE x.xx ALTER COLUMN StartTime DATETIME2 NOT NULL\nALTER TABLE x.xx ALTER COLUMN EndTime DATETIME2 NOT NULL\n'
    s = s + 'ALTER TABLE x.xx ADD PERIOD FOR SYSTEM_TIME (StartTime, EndTime)\n'
    s = s + 'ALTER TABLE x.xx SET(SYSTEM_VERSIONING = ON (HISTORY_TABLE = x.xx_History, DATA_CONSISTENCY_CHECK = ON))'
    sTemporal = s    

    s = 'ALTER TABLE x.xx ADD [ID] INT IDENTITY(1,1)'  #insert after .index('(')
    sID = s

    s = 'ALTER TABLE x.xx ADD CONSTRAINT [PK_x.xx] PRIMARY KEY CLUSTERED( [BKEY] ASC ),\n'
    sPK = s

In [4]:
# Classes setup

fwFS = fs()

In [5]:
lOps = [
            #[  'Source',        [ '%Product%',      '%PK%', 'i' ],      [ 'MatchOn', 'keyRegTS', 'dbo' ] ]   
            [  'Source',  
                [ 'Product%',   'SalesLT',      'seti' ],             # [2] s = stage table, e = etl sql, t = table, i = index
                [ 'MatchTable', 'NR',           '' ],                # [1] TABLE_NAME = same table must already exist in tgt, 'NR' = Matching not required/skip
                [ 'PKexcl',     'NameContains', ['guid'] ],          # [2] Discard PK if contains any of list in column_name
                [ 'IncrCol',    'ColContains',  'ModifiedDate' ]
            ]  
            #[  'Source',  [ '%Product%',  'SalesLT', 'ti' ],  [ 'MatchTableOn', 'TABLE_NAME', 'dbo' ],  [ 'PKexcl', 'Contains', ['ID','guid'] ] ]  
            #[ 'Manual',          [ 'stg.Customer',    '[CustomerID]', 'i' ] ],
            #[ 'Temporal',      [ 'stg.Customer',    '[CustomerID]', 't' ] ],
            #[ 'Temporal',      [ 'dbo.items',       '[item_nbr]', 't' ] ],
            #[ 'Temporal',      [ 'dbo.holidays',    '[date],[type],[locale],[locale_name],[description]', 't' ] ],
            ]

In [7]:
for i in lOps:

    if i[0] =='Temporal':
 
        if opType == 'Create':    # Template Setup
                    
            #Need to get BKey first
            #dbOps.genMergeScript(fwPath, i[1], i[2]) 

            fp = fwFS.getFileByPattern(pthRoot = pthFWroot, dbs = fsDB, objName = i[0], latest = True)   

            s = open(fp,'r').read()
            sObj = i[1][0] + sTemporalSuffix
            sAppend = sTemporal.replace('x.xx', sObj).replace('[BKEY]', i[1][1])
            
            sNew = sDrop.replace('x.xx', sObj) + s[:-4].replace('](\n', f'{sTemporalSuffix}](\n{sID}').replace('IDENTITY(1,1) ','') + sAppend

            fwPath = fwFS.genFlywayFullPath(pthRoot = pthFWroot, dbs = fwStage, objName = sObj, fleType = 't', opType = opType)


        elif opType == 'Alter':
            sAppend = sTemporal.replace('x.xx', i[1][0])

    elif i[0] =='Manual':
            
        sObj = sBKpattern.replace('x_xx', i[1][0].replace('.', '_') )
        sNew = sBK.replace('x.xx', i[1][0]).replace(sBKpattern, sObj).replace('[BKEY]', i[1][1])

    elif i[0] =='Source':

        dbTgt = db(lTgtDB[0], lTgtDB[1], lTgtDB[2], lTgtDB[3], lTgtDB[4]) 
        dbSrc = db(lSrcDB[0], lSrcDB[1], lSrcDB[2], lSrcDB[3], lSrcDB[4])

        lSrcTbls = dbSrc.getMatchingST( objName = i[1][0], schName = i[1][1] )

        for k in lSrcTbls:
 
            lSrcBKEYs = dbSrc.getIndexes( objName = k.table_name, schName = k.table_schem )
            jTbl = ''

            for j in lSrcBKEYs:
                
                if j.table_name != jTbl and j.index_qualifier == j.table_name:   # multi rows can be returned per table but we only need to do this loop once per table
                                                                                # need second check to exclude junk row from pyodbc
                    # 1 table exists in Tgt
                    matchRequired = (i[2][1] == 'TABLE_NAME')

                    jTbl = j.table_name
                    if matchRequired:
                        jrowTbl = dbTgt.getMatchingST( objName = j.table_name, schName = i[2][2] ) 
                    else:  #elif i[2][1] == 'Any'
                        jrowTbl = [k]

                    if 't' in i[1][2] and len(jrowTbl) > 0:
                        
                        tblCols = dbSrc.getCols( objName = j.table_name, schName = j.table_schem)

                        if 's' in i[1][2]:
                            sObj = 'stg.' + jrowTbl[0].table_name

                    #need to pass in both stg and src schema for use in etl sql schema 
                            tblDef = dbSrc.getTableDef( objName = j.table_name, schName = 'stg', stageTable = True, etlSchema = j.table_schem)
                            fwPath = fwFS.genFlywayFullPath(pthRoot = pthFWroot, dbs = fwStage, objName = sObj, fleType = 't', opType = opType)
                            open(fwPath, 'w').write(tblDef)

                            if 'e' in i[1][2]:
                                sObj = 'etl_' + sObj
                                fwPath = fwFS.genFlywayFullPath(pthRoot = pthFWroot, dbs = fwStage, objName = sObj, fleType = 't', opType = opType)
                                open(fwPath, 'w').write(dbSrc.getEtlSELECT())


                        sObj = i[2][2] + '.' + jrowTbl[0].table_name
                        tblDef = dbSrc.getTableDef( objName = j.table_name, schName = i[2][2])
                        fwPath = fwFS.genFlywayFullPath(pthRoot = pthFWroot, dbs = fwStage, objName = sObj, fleType = 't', opType = opType)
                        open(fwPath, 'w').write(tblDef)

                    if 'i' in i[1][2]:  # and len(jrowTbl) > 0

                        # 2 find Optimal BKEY in Src
                        jlstIdx = dbSrc.getIndexOptimalBKEY( objName = j.table_name, schName = j.table_schem, lstPKexclude = i[3][2] )
                        # jlstIdxALL.append(jlstIdx)

                        if len(jlstIdx) > 0:

                            # 3 all columns exists in Tgt table, pass in SRC
                            sBKEYcols = dbTgt.hasMatchingCols( objName = jrowTbl[0].table_name, schName = jrowTbl[0].table_schem, srcLstIndexCols = jlstIdx, matchRequired = matchRequired )

                            if sBKEYcols != '':
                                
                                #sBKpattern = 'uix_x_xx_BKEY'
                                #sBK = f'CREATE UNIQUE INDEX {sBKpattern} ON x.xx( [BKEY] ASC ) \n' 
                                sObj = sBKpattern.replace( 'x_xx', jrowTbl[0].table_schem + '_' + jrowTbl[0].table_name )
                                sNew = sBK.replace('x.xx', jrowTbl[0].table_schem + '.' + jrowTbl[0].table_name).replace(sBKpattern, sObj).replace('[BKEY]', sBKEYcols)  

                                # alternative is to use index name from src, ie objName = jlstIdx[0].index_name
                                fwPath = fwFS.genFlywayFullPath(pthRoot = pthFWroot, dbs = fwStage, objName = sObj, fleType = 'i', opType = opType)
                                open(fwPath, 'w').write(sNew)
